In [3]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
# .env 파일 로드 확인
load_dotenv('.env', verbose=True)

hf_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

login(hf_token)


In [4]:
import faiss

from sentence_transformers import SentenceTransformer
import json

# ✅ 문장 임베딩 모델 로드
embedding_model = SentenceTransformer("BM-K/KoSimCSE-roberta")

# ✅ 책 데이터 로드
with open("filtered_novels.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

# ✅ 책 요약을 임베딩 변환
book_summaries = [book["description"] for book in books_data]
book_titles = [book["title"] for book in books_data]

# 🔹 임베딩 변환
book_embeddings = embedding_model.encode(book_summaries, convert_to_numpy=True)

# 🔹 L2 정규화 (코사인 유사도 적용)
faiss.normalize_L2(book_embeddings)  # 벡터 정규화

# 🔹 FAISS 인덱스 생성 (L2 정규화된 벡터 기반)
dimension = book_embeddings.shape[1]  # 벡터 차원 (768)
faiss_index = faiss.IndexFlatIP(dimension)  # 🚀 코사인 유사도 적용 (Inner Product)
faiss_index.add(book_embeddings)  # 벡터 추가

# ✅ FAISS 인덱스 저장
faiss.write_index(faiss_index, "book_faiss_cosine_index.bin")

print("✅ 코사인 유사도를 적용한 FAISS 인덱스가 저장되었습니다.")


No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with mean pooling.


✅ 코사인 유사도를 적용한 FAISS 인덱스가 저장되었습니다.


In [5]:
import faiss
import json
from sentence_transformers import SentenceTransformer

# ✅ FAISS 인덱스 로드
faiss_index = faiss.read_index("book_faiss_cosine_index.bin")

# ✅ 책 데이터 로드
with open("filtered_novels.json", "r", encoding="utf-8") as f:
    books_data = json.load(f)

book_titles = [book["title"] for book in books_data]
book_summaries = [book["description"] for book in books_data]

# ✅ 문장 임베딩 모델 로드
embedding_model = SentenceTransformer("BM-K/KoSimCSE-roberta")


# ✅ FAISS 검색 함수
def find_similar_books(user_story, top_k=10):
    """유저 입력 스토리 → 코사인 유사도 기반 책 검색"""
    # 🔹 유저 입력을 문장 임베딩 변환
    user_embedding = embedding_model.encode([user_story], convert_to_numpy=True)
    faiss.normalize_L2(user_embedding)  # 벡터 정규화

    # 🔹 FAISS 검색
    distances, indices = faiss_index.search(user_embedding, top_k)

    # 🔹 검색된 책 목록 출력
    recommended_books = []
    for i in range(top_k):
        idx = indices[0][i]
        title = book_titles[idx]
        summary = book_summaries[idx]
        recommended_books.append(f"📖 **{title}**\n📌 {summary}\n")

    return recommended_books
# ✅ 테스트 실행
user_story = "전 남편을 살해한 여자의 죄를 덮기 위해 그녀를 사모하던 수학교사가 알리바이를 꾸미는 이야기"


print(find_similar_books(user_story, top_k=10))


No sentence-transformers model found with name BM-K/KoSimCSE-roberta. Creating a new one with mean pooling.


['📖 **밀랍 인형**\n📌 영국추리작가협회 실버대거상 수상작\n인형처럼 아름다운 그녀가 정말로 사람을 죽인 독살범일까?제한 시간이 주는 서스펜스 속에서 예리하게 빛나는 본격 추리!\n\n상류사회에서 인정받는 유명 사진사의 젊고 아름다운 아내가 남편의 조수를 독살한다. 순조로운 경찰 조사와 그녀의 자백으로 사건은 쉽게 마무리되는 듯했는데……. 사형선고가 내려진 직후, 런던 경찰청에 전달된 한 장의 사진이 모든 살해 정황을 부정하고 만다. 과연 그녀는 인형처럼 아름다운 독살범일까, 혹은 다른 이의 범행을 감춰주고 있는 무고한 여성일까?\n', '📖 **미스터 쉐도우 (정명섭 장편소설)**\n📌 딸의 죽음에 절망한 아버지의 복수를 돕는 미스터리한 킬러의 통쾌한 응징극\n\n일찍이 아내를 잃은 박기태의 인생은 하나뿐인 딸마저 갑작스럽게 죽었다는 소식에 산산조각 났다. 딸의 자살을 받아들이지 못하고 방황하던 그는 더욱 충격적인 진실과 마주하게 된다. 살인이 자살로 위장되었고, 그 뒤에는 교내 성매매 조직의 잔혹한 범죄가 숨겨져 있었던 것이다. 가해자들은 모두 권력층 자녀들이라 사건은 은폐되었고, 진실은 점점 더 깊은 어둠 속으로 묻혀 갔다.\n절망과 분노로 무너져 내린 박기태 앞에 어느 날 정체불명의 남자가 나타난다. ‘그림자’라 불리는 이 킬러는 박기태의 딸을 죽음으로 몰아넣은 자들을 응징하고 그 대가로 박기태의 목숨을 가져가기로 한다.\n그때부터 성매매 조직의 가해자들이 하나둘 의문의 죽음을 맞이하기 시작한다. 익사, 감전사, 도피한 해외에서 갱단에 의한 죽음. 모든 살인은 완벽하게 위장된다. 경찰청장에게서 이 연쇄적인 죽음의 배후를 파헤치라는 임무를 받은 전직 정보 경찰 권성호는 은밀하게 수사를 진행하고, 그들이 어떤 사건과 깊은 연관이 있다는 사실을 발견하게 되는데…….\n', '📖 **나는 그녀를 모른다 (로지 월쉬 장편소설)**\n📌 데뷔 소설 『전화하지 않는 남자 사랑에 빠진 여자』가 전 세계 35개 언어로 번역되어 150만 부 이상의 판매고를 올리면서